In [1]:
import pandas as pd

In [2]:
# import data
data = pd.read_csv(r"preprocessedData\hdgesamt_nodup.csv", sep=",")

In [3]:
# generate "start" and "end" OPS-Entry
data_noOPS = data[['Fall_ID', 'Pat_ID', 'Geschlecht', 'OPS_Altergruppe', 'ICD', 'ICDlabel'
                                        , 'Verweildauer']].drop_duplicates().reset_index() # create df without ops
startRow = pd.DataFrame({'OPSlabel': ['S'],'OPS_Datum_rel': [0],'rank_ratio': [-0.1],'rank': [0], 'rank_min': [0]}) #to create first row
endRow = pd.DataFrame({'OPSlabel': ['E'],'OPS_Datum_rel': [2000],'rank_ratio': [1.1],'rank': [1], 'rank_min': [1]})
startLabel = data_noOPS.join(startRow, how='left', ) # join with new row
startLabel.OPSlabel = 'S'
startLabel.OPS_Datum_rel = 0 
startLabel.rank_ratio = -0.1 # stricly smaller than 0.0
startLabel['rank'] = 0
startLabel.rank_min = 0 # set values of "start" to default, 
# do the same thing to "end"
endLabel = data_noOPS.join(endRow, how='left')
endLabel.OPSlabel = 'E'
endLabel.OPS_Datum_rel = 2000
endLabel.rank_ratio = 1.1 # stricly greater than 1.0
endLabel['rank'] = 1
endLabel.rank_min = 1
# finally concatenate data with the new created table
datanew = pd.concat([data,startLabel,endLabel], axis=0)
datanew = datanew.drop('index',axis=1) # drop the 'index' column
datanew = datanew.sort_values(by=['Fall_ID','rank_ratio','OPSlabel'],
        ascending=[True, True, True]).reset_index(drop=True) # sorted value

In [4]:
datanew['OPSprev'] = datanew['OPSlabel'].shift(1)
datanew['OPSprev'].replace('E', 'noprev', inplace=True)
datanew['OPSprev'][0] = 'noprev'

C:\Users\tyto4\AppData\Local\Temp\ipykernel_13556\4235458705.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['OPSprev'][0] = 'noprev'


In [5]:
datanew

,Fall_ID,Pat_ID,Geschlecht,OPS_Altergruppe,ICD,ICDlabel,OPSlabel,OPS_Datum_rel,Verweildauer,rank_ratio,rank,rank_min,OPSprev
0,2198995,709117,w,0,Q93.5,Q93.5,S,0,1008,-0.100000,0,0,noprev
1,2198995,709117,w,1,Q93.5,Q93.5,S,0,1008,-0.100000,0,0,S
2,2198995,709117,w,0,Q93.5,Q93.5,8-831,1,1008,0.000992,1,1,S
3,2198995,709117,w,0,Q93.5,Q93.5,8-800,2,1008,0.001984,1,1,8-831
4,2198995,709117,w,0,Q93.5,Q93.5,8-390,3,1008,0.002976,1,1,8-800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174821,4122526,213945,m,8,A09.0,A09.0,8-930,0,1,0.000000,0,0,S
174822,4122526,213945,m,8,A09.0,A09.0,E,2000,1,1.100000,1,1,8-930
174823,4123258,203628,m,6,I20.9,I20.9,S,0,0,-0.100000,0,0,noprev
174824,4123258,203628,m,6,I20.9,I20.9,8-930,0,0,0.000000,0,0,S


In [6]:
# Save as csv
datanew.to_csv(index=False,path_or_buf=r"preprocessedData\\hdgesamt_seq.csv")